In [1]:
import pandas as pd
import numpy as np
import sktime
import seaborn as snsc
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags
from visualization import graphs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sktime.datatypes import convert_to
from sktime.datatypes import MTYPE_REGISTER
from collections import Counter
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.compose import ClassifierPipeline
from storage.retrieve_csv import retrieve_csv
from storage.store_csv import store_csv

In [2]:
X, y = retrieve_csv("scaled_and_trimmed")
variances = pd.read_json("./storage/sorted_variance.json",orient="index")
variances.size

32

In [56]:
class Try_Classifiers:
    #Then in here, only classifiers should be needed. All classifiers implement fit and fit_transform as well as fit_predict.
    #If there are multiple classes that passed in, make the pipeline fit_transform -> fit_predict. Otherwise just fit_predict.
    
    def __init__(self, X_train:list, y_train=None, X_test=None, y_test=None):
        self.formats = [self.format_nested]
        # self.X_train_cols = X_train.columns
        # self.X_train_index = X_train.index
        self.X_train = X_train
        if y_train is not None:
            self.y_train = y_train
        if X_test is not None:
            self.X_test = X_test
        if y_test is not None:
            self.y_test = y_test


    def _try_function_with_formats(self, class_function, X, y=None):
        latestexception:Exception
        try:
            result = None
            if y is not None:
                result = class_function(X, y)
            else:
                result = class_function(X)
            return result, True
        except Exception as e:    
            latestexception = e   
            result = None   
            for fmt in self.formats:
                try:
                    formatted_X = fmt(X)
                    result = None
                    if y is not None: 
                        result = class_function(formatted_X, y)
                        return result, True
                        
                    else:
                        result = class_function(formatted_X)
                    return result, True
                except Exception as e:
                    latestexception = e
                    continue
            print(latestexception)
            return result, True
    
    def format_dflist(self, input):
        return convert_to(input, to_type="df-list")

    def format_multiindex(self, input):
        return convert_to(input, to_type="pd-multiindex")

    def format_nested(self, input):
        series_list = [pd.Series([df.to_numpy()]) for df in dfs]
        nested_df = pd.concat(series_list, axis=1).T
        nested_df.columns = ["time_series"]
        return nested_df


    def format_multiindex_manual(self, input):
        return pd.DataFrame(input, columns=self.X_train_cols, index=self.X_train_index)
    
    def format_multiindex_dflist(self, input):
        return convert_to(pd.DataFrame(input, columns=self.X_train_cols, index=self.X_train_index), to_type="df-list")

    def run_fit_predict_single(self, class_to_use):
        
        result, completed = self._try_function_with_formats(class_to_use.fit,self.X_train, self.y_train)
        
        if not completed:
            return Exception("Couldn't fit")

        if hasattr(class_to_use, 'score'):
            score, completed = self._try_function_with_formats(class_to_use.predict, self.X_test, self.y_test)
        else:
            prediction, completed = self._try_function_with_formats(class_to_use.predict, self.X_test)
            score = np.mean(prediction == self.y_test)
            
        
        if not completed:
            return Exception("Something went wrong")

#It runs the local fit function before sending it to the loop that catches exceptions
#The way to fix this is make sure that the fit and predict functions happen inside the loop. It has to handle multiple inputs or just one


In [3]:
def calculate_variance(df:pd.DataFrame) ->pd.Series:
    #The coefficient of variation
    cv = df.std() / df.mean()
    return cv.round(10).sort_values(ascending=False)

In [4]:
def select_by_variance(df:pd.DataFrame, amount:int):
    variances:pd.Series = calculate_variance(df)
    selection = variances.iloc[0:amount]
    return selection.index
best_features = select_by_variance(X, 3)
X = X[best_features]


In [51]:

# df_list = [group.reset_index(level=0,drop=True) for _, group in X.groupby(level=0)]

# # Convert each dataframe to have a multiindex
# X_train = [df.assign(outer=i).set_index(['outer', df.index]) for i, df in enumerate(X_train)]
# X_train = pd.concat(X_train)
# X_test = [df.assign(outer=i).set_index(['outer', df.index]) for i, df in enumerate(X_test)]
# X_test = pd.concat(X_test)

outer_index = X.index.get_level_values(0).unique()
#X.xs(1, level='instances',drop_level=True)
df_list = []

for level in outer_index:
    df_list.append(X.loc[level])


X_train, X_test, y_train, y_test = train_test_split(df_list, y)


In [49]:
#In this one, let's run with only the top 3. Run through many predictors.
# Importing necessary libraries
import numpy as np
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.transformations.panel.compose import ColumnConcatenator
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

# Initiate sktime classifiers
tsf = TimeSeriesForestClassifier()
boss = BOSSEnsemble()

# A multivariate time series can be thought of as a table of univariate time series.
# sktime's ColumnEnsembleClassifier treats each column (univariate time series) as a separate classification task and then aggregates predictions.
# Here we demonstrate it with TSF, but you can replace it with other sktime classifiers.
column_ens = ColumnEnsembleClassifier(estimators=[
    ("TSF0", TimeSeriesForestClassifier(), [0]),
    ("TSF1", TimeSeriesForestClassifier(), [1])
    # ... you can add more columns as necessary
])

# For sklearn classifiers, you'd typically need to transform the time series data into a tabular format.
# Here's an example using RandomForestClassifier from sklearn with ColumnConcatenator from sktime:
concatenator = ColumnConcatenator()
sklearn_classifier = make_pipeline(concatenator, RandomForestClassifier())

# List of all classifiers
classifiers = [tsf, boss,  column_ens, sklearn_classifier]

# You can now loop through `classifiers` to train and evaluate them.
looper = Try_Classifiers(X_train, y_train, X_test, y_test)


In [62]:
def format_nested(input):
    # Convert list of DataFrames to a truly nested DataFrame
    #series_list = [pd.Series([df.to_numpy()]) for df in input]

# Create a truly nested DataFrame
    series_list = [df.to_numpy() for df in input]
    nested_df = pd.DataFrame(series_list, columns=["series"])


    # Train classifier
    clf = TimeSeriesForestClassifier()
    clf.fit(nested_df, y)

tsf = TimeSeriesForestClassifier()
nested_xtrain = format_nested(X_train)
tsf.fit(nested_xtrain, y_train)

TypeError: X is not of a supported input data type.X must be in a supported mtype format for Panel, found <class 'pandas.core.series.Series'>Use datatypes.check_is_mtype to check conformance with specifications.

In [57]:
looper._try_function_with_formats(tsf.fit_predict, X_train, y_train)

X is not of a supported input data type.X must be in a supported mtype format for Panel, found <class 'list'>Use datatypes.check_is_mtype to check conformance with specifications.


(None, False)